In [7]:
import scipy
from scipy.optimize import curve_fit
from scipy.io import wavfile
from scipy.signal import butter, sosfiltfilt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal
import osascript
from gtts import gTTS 
from matplotlib import pylab
import os 
import pyaudio
import wave
import keyboard  # using module keyboard
import soundfile as sf
import math
import pyloudnorm as pyln
from sys import byteorder
from array import array
import noisereduce as nr
from struct import pack
import import_ipynb
import concat_project7
from joblib import dump, load
from emotion_recognition import EmotionRecognizer
from sklearn.svm import SVC
from deep_emotion_recognition import DeepEmotionRecognizer

/usr/local/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


importing Jupyter notebook from concat_project7.ipynb


Using TensorFlow backend.
W0828 14:46:31.592521 140736179348352 deprecation_wrapper.py:119] From /Users/m.elabd/Desktop/loudness/Emote-X/emotion-recognition-using-speech/deep_emotion_recognition.py:15: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



# Constants

In [2]:
THRESHOLD = 100
CHUNK_SIZE = 2048
FORMAT = pyaudio.paInt16
RATE = 16000
AVG_STEP = 75
PERCENTILE = 70
BANDPASS_FREQ = [300, 3400]
CALIBRATION_VOLUME = 40

# Record Audio

#### Record from microphone

In [3]:
def record():
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT, channels=1, rate=RATE,
        input=True, output=True,
        frames_per_buffer=CHUNK_SIZE)

    num_silent = 0
    snd_started = False

    r = array('h')

    while 1:
        # little endian, signed short
        snd_data = array('h', stream.read(CHUNK_SIZE))
        if byteorder == 'big':
            snd_data.byteswap()
        r.extend(snd_data)

        silent = is_silent(snd_data)

        if silent and snd_started:
            num_silent += 1
        elif not silent and not snd_started:
            snd_started = True

        if snd_started and num_silent > 60:
            break

    sample_width = p.get_sample_size(FORMAT)
    stream.stop_stream()
    stream.close()
    p.terminate()

#     r = trim(r)
#     r = add_silence(r, 0.5)
    return sample_width, r

#### Save to wav file

In [4]:
def record_to_file(path):
    sample_width, data = record()
    data = pack('<' + ('h'*len(data)), *data)

    wf = wave.open(path, 'wb')
    wf.setnchannels(1)
    wf.setsampwidth(sample_width)
    wf.setframerate(RATE)
    wf.writeframes(data)
    wf.close()

#### Checks if input is silent based on threshold

In [5]:
def is_silent(snd_data):
    "Returns 'True' if below the 'silent' threshold"
    return max(snd_data) < THRESHOLD

# Preprocessing

#### Trim input to remove silence at beginning and end

In [6]:
def trim(snd_data):
    def _trim(snd_data):
        snd_started = False
        r = array('h')

        for i in snd_data:
            if not snd_started and abs(i)>THRESHOLD:
                snd_started = True
                r.append(i)
            elif snd_started:
                r.append(i)
        return r

    # Trim to the left
    snd_data = _trim(snd_data)

    # Trim to the right
    snd_data.reverse()
    snd_data = _trim(snd_data)
    snd_data.reverse()
    return snd_data

#### Pad with silence at beginning and end

In [7]:
def add_silence(snd_data, seconds):
    ""
    r = array('h', [0 for i in range(int(seconds*RATE))])
    r.extend(snd_data)
    r.extend([0 for i in range(int(seconds*RATE))])
    return r

#### Process wav input with highpass filter

In [8]:
def voice_input(voice_data):
    fs, data = wavfile.read(voice_data)
    data = butter_bandpass_filter(data, BANDPASS_FREQ[0], BANDPASS_FREQ[1], fs)
    return data, fs

#### Implement butter bandpass filter

In [9]:
# def butter_bandpass(lowcut, highcut, fs, order=5):
#     nyq = 0.5 * fs
#     low = lowcut / nyq
#     high = highcut / nyq
#     sos = butter(order, [low, high], btype='band', analog=False, output='sos')
#     return sos

# def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
#     sos = butter_bandpass(lowcut, highcut, fs, order=order)
#     y = sosfiltfilt(sos, data)
#     return y

# Feature Extraction

# Emotion Algorithm

#### Get amplitude curve and apply regression on ITU-R BS.1770-4 loudness algorithm

In [10]:
# def getting_emotion(data, rate):

    
# #     lol_data, fs = concat_project7.clean_sound(data, rate)

#     #Convert raw data into features
#     print("Data acquired")

#     feature_data = concat_project7.get_all_features(data, fs);
    
#     #return the exponential regression
#     return (feature_data)

#### Remove silence before, after, and in-between voice data

In [11]:
def remove_silence_from(amplitudes, threshold):
    silenced = []
    for x in amplitudes:
        if x >= threshold:
            silenced.append(x)
    return silenced# Preprocessing

#### Regression converts loudness to scale 0 to 100 and adjusts for calibration offset

# Calibration

In [12]:
# def get_offset(calib_loudness):
#     return calib_loudness - CALIBRATION_VOLUME

# Response

#### Sets a new volume that is spoken at for the user

In [3]:
def reply_volume(user_volume):
    speaker_volume = int(osascript.osascript('output volume of (get volume settings)')[1])
    print("speaker volume is " + str(speaker_volume))
    print("user volume is "+ str(user_volume))
    if(speaker_volume < user_volume):
        new_volume = speaker_volume + (((speaker_volume - user_volume)**2) /100)
    else:
        new_volume = speaker_volume - (((speaker_volume - user_volume)**2) /100)
    print("speaker new volume is " + str(new_volume))
    osascript.osascript("set volume output volume "+ str(new_volume)) 
    thing_to_say = 'hello your volume is at '+ str(int(new_volume))
    print(thing_to_say)
    os.system("say "+ thing_to_say)

In [4]:
def reply_emotion(user_emotion):
    os.system("say "+" are you feeling "+user_emotion)

# Main

In [15]:
deeprec = DeepEmotionRecognizer(emotions=['sad', 'neutral', 'happy', 'angry', 'disgust', 'fear'],
                                n_rnn_layers=2, n_dense_layers=3, rnn_units=256, dense_units=128,
                                dropout=0.35,epochs=500)
# train the model
deeprec.train(override=False)
# get the accuracy
print(deeprec.test_score())
print(deeprec.confusion_matrix())

[TESS&RAVDESS] There are 812 training audio files for category:sad
[TESS&RAVDESS] There are 146 testing audio files for category:sad
[TESS&RAVDESS] There are 585 training audio files for category:neutral
[TESS&RAVDESS] There are 93 testing audio files for category:neutral
[TESS&RAVDESS] There are 805 training audio files for category:happy
[TESS&RAVDESS] There are 147 testing audio files for category:happy
[TESS&RAVDESS] There are 808 training audio files for category:angry
[TESS&RAVDESS] There are 147 testing audio files for category:angry
[TESS&RAVDESS] There are 511 training audio files for category:disgust
[TESS&RAVDESS] There are 79 testing audio files for category:disgust
[TESS&RAVDESS] There are 817 training audio files for category:fear
[TESS&RAVDESS] There are 141 testing audio files for category:fear
[+] Writed TESS & RAVDESS DB CSV File
[EMO-DB] Total files to write: 454
[EMO-DB] Training samples: 363
[EMO-DB] Testing samples: 90
[+] Writed EMO-DB CSV File


Extracting features for train:   0%|          | 2/10821 [00:00<15:45, 11.44it/s]

[+] Writed Custom DB CSV File


Extracting features for train:   1%|          | 110/10821 [00:06<06:18, 28.32it/s]


KeyboardInterrupt: 

In [ ]:
# os.system("say "+ "Please speak into the microphone")
# print("Please speak into the microphone")
# record_to_file('emotion.wav')
# print("done")
# prediction = deeprec.predict('emotion.wav')
# print(f"Prediction: {prediction}")
# reply_emotion(prediction)


In [ ]:
from emotion_recognition import EmotionRecognizer
from sklearn.svm import SVC
# init a model, let's use SVC
my_model = SVC()
# pass my model to EmotionRecognizer instance
# and balance the dataset
rec = EmotionRecognizer(model=my_model, emotions=['angry', 'happy', 'sad'], balance=True, verbose=0)
# train the model
rec.train()
# check the test accuracy for that model
print("Test score:", rec.test_score())
# check the train accuracy for that model
print("Train score:", rec.train_score())

In [ ]:
# loads the best estimators from `grid` folder that was searched by GridSearchCV in `grid_search.py`,
# and set the model to the best in terms of test score, and then train it
rec.determine_best_model(train=True)
# get the determined sklearn model name
print(rec.model.__class__.__name__, "is the best")
# get the test accuracy score for the best estimator
print("Test score:", rec.test_score())

In [ ]:
# os.system("say "+ "Please speak into the microphone")
# print("Please speak into the microphone")
# record_to_file('emotion.wav')
# print("done")
# prediction = rec.predict('emotion.wav')
# print(f"Prediction: {prediction}")
# reply_emotion(prediction)


In [51]:
from emotion_recognition import EmotionRecognizer

import pyaudio
import os
import wave
from sys import byteorder
from array import array
from struct import pack
from sklearn.ensemble import GradientBoostingClassifier, BaggingClassifier

from utils import get_best_estimators

THRESHOLD = 500
CHUNK_SIZE = 1024
FORMAT = pyaudio.paInt16
RATE = 16000

SILENCE = 30

def is_silent(snd_data):
    "Returns 'True' if below the 'silent' threshold"
    return max(snd_data) < THRESHOLD

def normalize(snd_data):
    "Average the volume out"
    MAXIMUM = 16384
    times = float(MAXIMUM)/max(abs(i) for i in snd_data)

    r = array('h')
    for i in snd_data:
        r.append(int(i*times))
    return r

def trim(snd_data):
    "Trim the blank spots at the start and end"
    def _trim(snd_data):
        snd_started = False
        r = array('h')

        for i in snd_data:
            if not snd_started and abs(i)>THRESHOLD:
                snd_started = True
                r.append(i)

            elif snd_started:
                r.append(i)
        return r

    # Trim to the left
    snd_data = _trim(snd_data)

    # Trim to the right
    snd_data.reverse()
    snd_data = _trim(snd_data)
    snd_data.reverse()
    return snd_data

def add_silence(snd_data, seconds):
    "Add silence to the start and end of 'snd_data' of length 'seconds' (float)"
    r = array('h', [0 for i in range(int(seconds*RATE))])
    r.extend(snd_data)
    r.extend([0 for i in range(int(seconds*RATE))])
    return r

def record():
    """
    Record a word or words from the microphone and 
    return the data as an array of signed shorts.

    Normalizes the audio, trims silence from the 
    start and end, and pads with 0.5 seconds of 
    blank sound to make sure VLC et al can play 
    it without getting chopped off.
    """
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT, channels=1, rate=RATE,
        input=True, output=True,
        frames_per_buffer=CHUNK_SIZE)

    num_silent = 0
    snd_started = False

    r = array('h')

    while 1:
        # little endian, signed short
        snd_data = array('h', stream.read(CHUNK_SIZE))
        if byteorder == 'big':
            snd_data.byteswap()
        r.extend(snd_data)

        silent = is_silent(snd_data)

        if silent and snd_started:
            num_silent += 1
        elif not silent and not snd_started:
            snd_started = True

        if snd_started and num_silent > SILENCE:
            break

    sample_width = p.get_sample_size(FORMAT)
    stream.stop_stream()
    stream.close()
    p.terminate()

    r = normalize(r)
    r = trim(r)
    r = add_silence(r, 0.5)
    return sample_width, r

def record_to_file(path):
    "Records from the microphone and outputs the resulting data to 'path'"
    sample_width, data = record()
    # plt.plot(data)
#     noise_ms = 100 #how much noise at the beginning to sample (ms)
#     data = np.asarray(data)
#     clean = data.astype(float)
#     for i in range(2):
#         noise = clean[0:int(noise_ms*RATE/1000)]
#         clean = nr.reduce_noise(audio_clip=clean, noise_clip=noise, verbose=False)
#     #     ipd.Audio(clean, rate=fs) #do whatever you want with clean
#     data = clean.astype(int)

    data = pack('<' + ('h'*len(data)), *data)

    wf = wave.open(path, 'wb')
    wf.setnchannels(1)
    wf.setsampwidth(sample_width)
    wf.setframerate(RATE)
    wf.writeframes(data)
    wf.close()



def get_estimators_name(estimators):
    result = [ '"{}"'.format(estimator.__class__.__name__) for estimator, _, _ in estimators ]
    return ','.join(result), {estimator_name.strip('"'): estimator for estimator_name, (estimator, _, _) in zip(result, estimators)}





In [50]:
print("Please speak into the microphone")
record_to_file('emotion.wav')

Please speak into the microphone


# DEEP REC NN

In [6]:
if __name__ == "__main__":
    deeprec = DeepEmotionRecognizer(emotions=['sad', 'happy'], 
                                    features = ['mfcc', 'mel', 'contrast', 'chroma', 'tonnetz', 'deriv'],
                                    n_rnn_layers=2, n_dense_layers=3, rnn_units=256, dense_units=128,
                                    dropout=0.35,epochs=500, batch_size=512)
    # train the model
    deeprec.train(override=False)
    # get the accuracy
    print(deeprec.test_score())

    print("Test accuracy score: {:.3f}%".format(deeprec.test_score()*100))
#     print("Please talk")
    
    
#     filename = "test.wav"
#     record_to_file(filename)
#     result = deeprec.predict(filename)
#     print(result)
# NOTES
# look for happy vs anger



Extracting features for train:   0%|          | 0/3229 [00:00<?, ?it/s]

[TESS&RAVDESS] There are 508 training audio files for category:sad
[TESS&RAVDESS] There are 82 testing audio files for category:sad
[TESS&RAVDESS] There are 501 training audio files for category:happy
[TESS&RAVDESS] There are 83 testing audio files for category:happy
[+] Writed TESS & RAVDESS DB CSV File
[EMO-DB] Total files to write: 133
[EMO-DB] Training samples: 106
[EMO-DB] Testing samples: 26
[+] Writed EMO-DB CSV File
[+] Writed Custom DB CSV File


Extracting features for train:   8%|▊         | 254/3229 [01:20<29:49,  1.66it/s]

KeyboardInterrupt: 

In [19]:
deeprec.confusion_matrix()

,predicted_sad,predicted_happy
true_sad,88.616074,11.383928
true_happy,11.607142,88.392860


In [44]:
print("Please speak into the microphone")
record_to_file('emotion.wav')

Please speak into the microphone


In [11]:
RATE

16000

In [18]:
# os.system("say "+ "Please speak into the microphone")
print("Please speak into the microphone")
record_to_file('emotion.wav')

print("done")
prediction, confidence = deeprec.predict('emotion.wav')
print(f"Prediction: {prediction}")
reply_emotion(prediction)
print("happy confidence: " + str(confidence[0][0][1]*100) + "%")
print("sad confidence: " + str(confidence[0][0][0]*100) + "%")


Please speak into the microphone
done
{'mfcc': True, 'chroma': True, 'mel': True, 'contrast': True, 'tonnetz': True, 'rmse': False, 'poly_feature': False, 'chroma_cens': False, 'deriv': True}
(1, 1, 386)


ValueError: Error when checking input: expected lstm_1_input to have shape (None, 412) but got array with shape (1, 386)

In [3]:
if __name__ == "__main__":
    deeprec_angry = DeepEmotionRecognizer(emotions=['sad', 'happy', 'angry'],
                                    features = ['mfcc', 'mel', 'contrast', 'chroma','rmse','poly_feature','chroma_cens',
                                                'tonnetz', 'deriv'],
                                    n_rnn_layers=2, n_dense_layers=3, rnn_units=256, dense_units=128,
                                    dropout=0.35,epochs=500, batch_size=512)
    # train the model
    deeprec_angry.train(override=False)
    # get the accuracy
    print(deeprec_angry.test_score())

    print("Test accuracy score: {:.3f}%".format(deeprec_angry.test_score()*100))
#     print("Please talk")
    
    
#     filename = "test.wav"
#     record_to_file(filename)
#     result = deeprec.predict(filename)
#     print(result)
# NOTES
# look for happy vs anger
# api to remove other people voice


W0816 12:38:39.019542 140735676818304 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0816 12:38:39.031781 140735676818304 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



[TESS&RAVDESS] There are 812 training audio files for category:sad
[TESS&RAVDESS] There are 146 testing audio files for category:sad
[TESS&RAVDESS] There are 805 training audio files for category:happy
[TESS&RAVDESS] There are 147 testing audio files for category:happy
[TESS&RAVDESS] There are 808 training audio files for category:angry
[TESS&RAVDESS] There are 147 testing audio files for category:angry
[+] Writed TESS & RAVDESS DB CSV File
[EMO-DB] Total files to write: 260
[EMO-DB] Training samples: 208
[EMO-DB] Testing samples: 51
[+] Writed EMO-DB CSV File
[+] Writed Custom DB CSV File
[+] Data loaded


W0816 12:38:39.034219 140735676818304 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0816 12:38:39.245239 140735676818304 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0816 12:38:39.251039 140735676818304 deprecation.py:506] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0816 12:38:39.581840 140735676818304 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/optimize

[+] Model created
[*] Model weights loaded
0.7986960065199674
Test accuracy score: 79.870%


In [6]:
## os.system("say "+ "Please speak into the microphone")
print("Please speak into the microphone")
record_to_file('emotion.wav')
print("done")
prediction, confidence = deeprec_angry.predict('emotion.wav')
print(f"Prediction: {prediction}")
reply_emotion(prediction)
print("happy confidence: " + str(confidence[0][0][1]*100) + "%")
print("sad confidence: " + str(confidence[0][0][0]*100) + "%")
print("angry confidence: " + str(confidence[0][0][2]*100) + "%")


Please speak into the microphone
done
{'mfcc': True, 'chroma': True, 'mel': True, 'contrast': True, 'tonnetz': True, 'rmse': True, 'poly_feature': True, 'chroma_cens': True, 'deriv': True}
(1, 1, 412)


ValueError: Error when checking input: expected lstm_1_input to have shape (None, 386) but got array with shape (1, 412)

In [ ]:
if __name__ == "__main__":
    deeprec_angry = DeepEmotionRecognizer(emotions=['sad', 'happy', 'angry'],
                                    features = ['mfcc', 'mel', 'contrast', 'chroma','rmse','poly_feature','chroma_cens',
                                                'tonnetz', 'deriv'],
                                    n_rnn_layers=2, n_dense_layers=2, rnn_units=128, dense_units=128,
                                    dropout=0.35,epochs=500, batch_size=512)
    # train the model
    deeprec_angry.train(override=False)
    # get the accuracy
    print(deeprec_angry.test_score())

    print("Test accuracy score: {:.3f}%".format(deeprec_angry.test_score()*100))
#     print("Please talk")
    
    
#     filename = "test.wav"
#     record_to_file(filename)
#     result = deeprec.predict(filename)
#     print(result)
# NOTES
# look for happy vs anger
# api to remove other people voice


In [3]:
if __name__ == "__main__":
    deeprec = DeepEmotionRecognizer(emotions=['happy', 'sad'],
                        features=['mfcc', 'chroma','mel','contrast','tonnetz', 'rmse', 'poly_feature','chroma_cens','deriv'],
                                    balance=True, verbose=1,
                                    n_rnn_layers=2, n_dense_layers=3, rnn_units=128, dense_units=256,
                                    dropout=0.3,epochs=300)
    # train the model
    deeprec.train(override=True)
    # get the accuracy
    print(deeprec.test_score())

    print("Test accuracy score: {:.3f}%".format(deeprec.test_score()*100))
#     print("Please talk")
    
    
#     filename = "test.wav"
#     record_to_file(filename)
#     result = deeprec.predict(filename)
#     print(result)

[TESS&RAVDESS] There are 805 training audio files for category:happy
[TESS&RAVDESS] There are 147 testing audio files for category:happy
[TESS&RAVDESS] There are 812 training audio files for category:sad
[TESS&RAVDESS] There are 146 testing audio files for category:sad
[+] Writed TESS & RAVDESS DB CSV File
[EMO-DB] Total files to write: 133
[EMO-DB] Training samples: 106
[EMO-DB] Testing samples: 26
[+] Writed EMO-DB CSV File
[+] Writed Custom DB CSV File
[+] Data loaded
[+] Model created


W0815 15:21:56.064957 140736091374464 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 1 samples, validate on 1 samples


W0815 15:21:58.115002 140736091374464 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/callbacks.py:850: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0815 15:21:58.115922 140736091374464 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/callbacks.py:853: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Epoch 1/300
1/1 [==============================] - 7s 7s/step - loss: 0.7872 - acc: 0.5056 - val_loss: 1.3414 - val_acc: 0.5000

Epoch 00001: val_loss improved from inf to 1.34139, saving model to results/HS-c-LSTM-layers-2-3-units-128-256-dropout-0.3_0.3_0.3_0.3_0.3.h5
Epoch 2/300
1/1 [==============================] - 5s 5s/step - loss: 1.3605 - acc: 0.5003 - val_loss: 0.8130 - val_acc: 0.5000

Epoch 00002: val_loss improved from 1.34139 to 0.81296, saving model to results/HS-c-LSTM-layers-2-3-units-128-256-dropout-0.3_0.3_0.3_0.3_0.3.h5
Epoch 3/300
1/1 [==============================] - 5s 5s/step - loss: 0.8540 - acc: 0.5003 - val_loss: 0.6956 - val_acc: 0.5000

Epoch 00003: val_loss improved from 0.81296 to 0.69563, saving model to results/HS-c-LSTM-layers-2-3-units-128-256-dropout-0.3_0.3_0.3_0.3_0.3.h5
Epoch 4/300
1/1 [==============================] - 5s 5s/step - loss: 0.7265 - acc: 0.4989 - val_loss: 0.7642 - val_acc: 0.5000

Epoch 00004: val_loss did not improve from 0.69563

...


Epoch 00097: val_loss did not improve from 0.28519
Epoch 98/300
1/1 [==============================] - 6s 6s/step - loss: 0.2383 - acc: 0.9019 - val_loss: 0.2858 - val_acc: 0.8884

Epoch 00098: val_loss did not improve from 0.28519
Epoch 99/300
1/1 [==============================] - 5s 5s/step - loss: 0.2321 - acc: 0.8979 - val_loss: 0.2870 - val_acc: 0.8906

Epoch 00099: val_loss did not improve from 0.28519
Epoch 100/300
1/1 [==============================] - 5s 5s/step - loss: 0.2331 - acc: 0.9024 - val_loss: 0.2874 - val_acc: 0.8929

Epoch 00100: val_loss did not improve from 0.28519
Epoch 101/300
1/1 [==============================] - 5s 5s/step - loss: 0.2290 - acc: 0.9043 - val_loss: 0.2865 - val_acc: 0.8906

Epoch 00101: val_loss did not improve from 0.28519
Epoch 102/300
1/1 [==============================] - 5s 5s/step - loss: 0.2256 - acc: 0.9046 - val_loss: 0.2841 - val_acc: 0.8929

Epoch 00102: val_loss improved from 0.28519 to 0.28411, saving model to results/HS-c-LSTM-l


Epoch 00141: val_loss did not improve from 0.28252
Epoch 142/300
1/1 [==============================] - 5s 5s/step - loss: 0.1509 - acc: 0.9386 - val_loss: 0.3065 - val_acc: 0.9040

Epoch 00142: val_loss did not improve from 0.28252
Epoch 143/300
1/1 [==============================] - 5s 5s/step - loss: 0.1491 - acc: 0.9389 - val_loss: 0.3117 - val_acc: 0.9007

Epoch 00143: val_loss did not improve from 0.28252
Epoch 144/300
1/1 [==============================] - 5s 5s/step - loss: 0.1555 - acc: 0.9381 - val_loss: 0.3225 - val_acc: 0.8962

Epoch 00144: val_loss did not improve from 0.28252
Epoch 145/300
1/1 [==============================] - 5s 5s/step - loss: 0.1580 - acc: 0.9346 - val_loss: 0.3174 - val_acc: 0.9029

Epoch 00145: val_loss did not improve from 0.28252
Epoch 146/300
1/1 [==============================] - 6s 6s/step - loss: 0.1522 - acc: 0.9415 - val_loss: 0.3172 - val_acc: 0.9007

Epoch 00146: val_loss did not improve from 0.28252
Epoch 147/300
1/1 [===================

1/1 [==============================] - 5s 5s/step - loss: 0.1160 - acc: 0.9511 - val_loss: 0.3926 - val_acc: 0.8951

Epoch 00187: val_loss did not improve from 0.28252
Epoch 188/300
1/1 [==============================] - 5s 5s/step - loss: 0.1062 - acc: 0.9577 - val_loss: 0.3882 - val_acc: 0.8996

Epoch 00188: val_loss did not improve from 0.28252
Epoch 189/300
1/1 [==============================] - 5s 5s/step - loss: 0.1039 - acc: 0.9583 - val_loss: 0.3793 - val_acc: 0.8996

Epoch 00189: val_loss did not improve from 0.28252
Epoch 190/300
1/1 [==============================] - 5s 5s/step - loss: 0.1134 - acc: 0.9551 - val_loss: 0.3780 - val_acc: 0.8973

Epoch 00190: val_loss did not improve from 0.28252
Epoch 191/300
1/1 [==============================] - 5s 5s/step - loss: 0.1067 - acc: 0.9585 - val_loss: 0.3863 - val_acc: 0.8962

Epoch 00191: val_loss did not improve from 0.28252
Epoch 192/300
1/1 [==============================] - 5s 5s/step - loss: 0.1079 - acc: 0.9564 - val_loss:


Epoch 00232: val_loss did not improve from 0.28252
Epoch 233/300
1/1 [==============================] - 5s 5s/step - loss: 0.0830 - acc: 0.9678 - val_loss: 0.4654 - val_acc: 0.8817

Epoch 00233: val_loss did not improve from 0.28252
Epoch 234/300
1/1 [==============================] - 5s 5s/step - loss: 0.0855 - acc: 0.9689 - val_loss: 0.4325 - val_acc: 0.8884

Epoch 00234: val_loss did not improve from 0.28252
Epoch 235/300
1/1 [==============================] - 5s 5s/step - loss: 0.0967 - acc: 0.9596 - val_loss: 0.4302 - val_acc: 0.8973

Epoch 00235: val_loss did not improve from 0.28252
Epoch 236/300
1/1 [==============================] - 6s 6s/step - loss: 0.0819 - acc: 0.9670 - val_loss: 0.4745 - val_acc: 0.8817

Epoch 00236: val_loss did not improve from 0.28252
Epoch 237/300
1/1 [==============================] - 6s 6s/step - loss: 0.0918 - acc: 0.9644 - val_loss: 0.4268 - val_acc: 0.8984

Epoch 00237: val_loss did not improve from 0.28252
Epoch 238/300
1/1 [===================

1/1 [==============================] - 5s 5s/step - loss: 0.0649 - acc: 0.9763 - val_loss: 0.4792 - val_acc: 0.8951

Epoch 00278: val_loss did not improve from 0.28252
Epoch 279/300
1/1 [==============================] - 5s 5s/step - loss: 0.0663 - acc: 0.9710 - val_loss: 0.4826 - val_acc: 0.8940

Epoch 00279: val_loss did not improve from 0.28252
Epoch 280/300
1/1 [==============================] - 5s 5s/step - loss: 0.0654 - acc: 0.9755 - val_loss: 0.5012 - val_acc: 0.8850

Epoch 00280: val_loss did not improve from 0.28252
Epoch 281/300
1/1 [==============================] - 5s 5s/step - loss: 0.0703 - acc: 0.9755 - val_loss: 0.4820 - val_acc: 0.8951

Epoch 00281: val_loss did not improve from 0.28252
Epoch 282/300
1/1 [==============================] - 5s 5s/step - loss: 0.0672 - acc: 0.9737 - val_loss: 0.4733 - val_acc: 0.8940

Epoch 00282: val_loss did not improve from 0.28252
Epoch 283/300
1/1 [==============================] - 5s 5s/step - loss: 0.0700 - acc: 0.9732 - val_loss:

In [ ]:
os.system("say "+ "Please speak into the microphone")
print("Please speak into the microphone")
record_to_file('emotion.wav')
print("done")
prediction, confidence = deeprec.predict('emotion.wav')
print(f"Prediction: {prediction}")
reply_emotion(prediction)
print("netrual confidence: " + str(confidence[0][0][1]*100) + "%")
print("sad confidence: " + str(confidence[0][0][0]*100) + "%")
print("happy confidence: " + str(confidence[0][0][2]*100) + "%")


# ML algorithm

In [4]:
from emotion_recognition import EmotionRecognizer
from sklearn.svm import SVC
# init a model, let's use SVC
my_model = SVC()
# pass my model to EmotionRecognizer instance
# and balance the dataset
#     pitch_cepstrum = kwargs.get('pitch_cepstrum')
#     formants = kwargs.get('formants')
#     rmse = kwargs.get('rmse')
#     poly_features = kwargs.get('poly_features')
#     deriv = kwargs.get('deriv')
rec = EmotionRecognizer(model=my_model, emotions=['happy', 'sad'],
                        features=['mfcc', 'chroma','mel','contrast','tonnetz', 'deriv'],
                        balance=True, verbose=1)
# train the model
rec.train()
# check the test accuracy for that model
print("Test score:", rec.test_score())
# check the train accuracy for that model
print("Train score:", rec.train_score())

[TESS&RAVDESS] There are 805 training audio files for category:happy
[TESS&RAVDESS] There are 147 testing audio files for category:happy
[TESS&RAVDESS] There are 812 training audio files for category:sad
[TESS&RAVDESS] There are 146 testing audio files for category:sad
[+] Writed TESS & RAVDESS DB CSV File
[EMO-DB] Total files to write: 133
[EMO-DB] Training samples: 106
[EMO-DB] Testing samples: 26
[+] Writed EMO-DB CSV File
[+] Writed Custom DB CSV File
[+] Data loaded


/usr/local/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


[+] Model trained
Test score: 0.8738839285714286
Train score: 0.9718234981392876


In [5]:
# loads the best estimators from `grid` folder that was searched by GridSearchCV in `grid_search.py`,
# and set the model to the best in terms of test score, and then train it
rec.determine_best_model(train=True)
# get the determined sklearn model name
print(rec.model.__class__.__name__, "is the best")
# get the test accuracy score for the best estimator
print("Test score:", rec.test_score())

Evaluating BaggingClassifier: 100%|██████████| 6/6 [01:29<00:00, 15.40s/it]         

[+] Best model determined: GradientBoostingClassifier with 89.955% test accuracy
GradientBoostingClassifier is the best
Test score: 0.8995535714285714


In [ ]:
os.system("say "+ "Please speak into the microphone")
print("Please speak into the microphone")
record_to_file('emotion.wav')
print("done")
prediction = rec.predict('emotion.wav')
print(f"Prediction: {prediction}")
reply_emotion(prediction)

In [ ]:
if __name__ == "__main__":
    deeprec = DeepEmotionRecognizer(emotions=['sad', 'happy'],
                                    n_rnn_layers=2, n_dense_layers=3, rnn_units=256, dense_units=128,
                                    dropout=0.35,epochs=500)
    # train the model
    deeprec.train(override=False)
    # get the accuracy
    print(deeprec.test_score())

    print("Test accuracy score: {:.3f}%".format(deeprec.test_score()*100))
#     print("Please talk")
    
    
#     filename = "test.wav"
#     record_to_file(filename)
#     result = deeprec.predict(filename)
#     print(result)